# Azure AI Agent with Code Interpreter Example

This notebook demonstrates using HostedCodeInterpreterTool with Azure AI Agents for Python code execution and mathematical problem solving.

## Features Covered:
- Setting up a code interpreter tool for agents
- Executing Python code through the agent
- Mathematical problem solving
- Accessing code interpreter inputs and outputs
- Working with streaming responses

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Azure AI Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with connection details
4. **Dependencies**: Required agent-framework packages installed

If you need to use a different tenant, specify the tenant ID:
```bash
az login --tenant <tenant-id>
```

## Import Libraries

Import the required libraries for Azure AI agent functionality.

In [ ]:
#!about
#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.*"
#r "nuget: Microsoft.Agents.AI.AzureAI, 1.0.0-preview.*"
#r "nuget: dotenv.net"

#pragma warning disable CA1812
#pragma warning disable MEAI001

using Azure.Identity;
using Microsoft.Agents.AI;
using Azure.AI.Projects;
using Azure.AI.Projects.OpenAI;
using Microsoft.Extensions.AI;
using dotenv.net;
using System.IO;
using System.Threading.Tasks;
using System.ComponentModel;
using System.Text.Json;
using OpenAI.Assistants;
using OpenAI.Responses;

## Initial Setup

Initialize the Azure AI project client and set up authentication.

In [ ]:
DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".", "..", "..","..", ".env") })); 
var tenantId = Environment.GetEnvironmentVariable("TENANT_ID");
var projectEndpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT");
var modelDeploymentName = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME");
// Create the Credentials
var credentialOptions = new InteractiveBrowserCredentialOptions
{
    TenantId = tenantId
};
var credential = new InteractiveBrowserCredential(credentialOptions);

AIProjectClient aiProjectClient = new AIProjectClient(new Uri(projectEndpoint)!,credential);

## Helper Function for Code Interpreter Data

This helper function allows us to access and display the actual code that was executed by the code interpreter:

In [ ]:
#pragma warning disable MEAI001
void ShowToolCallContent(CodeInterpreterToolCallContent? toolCallContent)
{
    if (toolCallContent?.Inputs is not null)
    {
        DataContent? codeInput = toolCallContent.Inputs.OfType<DataContent>().FirstOrDefault();
        if (codeInput?.HasTopLevelMediaType("text") ?? false)
        {
            Console.WriteLine($"Code Input: {Encoding.UTF8.GetString(codeInput.Data.ToArray()) ?? "Not available"}");
        }
    }
}

void ShowToolCallResultContent(CodeInterpreterToolResultContent? toolResultContent)
{
    if (toolResultContent?.Outputs is not null && toolResultContent.Outputs.OfType<TextContent>().FirstOrDefault() is { } resultOutput)
    {
        Console.WriteLine($"Code Tool Result: {resultOutput.Text}");
    }
}

## Create an Agent Run

Create a method that takes an agent and a query for making it reusable.

In [ ]:
#pragma warning disable MEAI001
#pragma warning disable OPENAI001
async Task RunAgentWithQuery(AIAgent agent, string query)
{
    var response = await agent.RunAsync(query);
    Console.WriteLine($"🤖 Agent: {response}");
    var toolCallContent = response.Messages
        .SelectMany(m => m.Contents)
        .OfType<CodeInterpreterToolCallContent>()
        .FirstOrDefault();
    ShowToolCallContent(toolCallContent);

    var toolResultContent = response.Messages
        .SelectMany(m => m.Contents)
        .OfType<CodeInterpreterToolResultContent>()
        .FirstOrDefault();
    ShowToolCallResultContent(toolResultContent);

    // Get any annotations generated by the tool
    foreach (var annotation in response.Messages.SelectMany(m => m.Contents).SelectMany(c => c.Annotations ?? []))
    {
        if (annotation.RawRepresentation is TextAnnotation citationAnnotation)
        {
            Console.WriteLine($$"""
                File Id: {{citationAnnotation.OutputFileId}}
                Text to Replace: {{citationAnnotation.TextToReplace}}
                Filename: {{Path.GetFileName(citationAnnotation.TextToReplace)}}
                """);
        }
    }
}

## Create and Run the Code Interpreter Agent

Now let's create an agent that can write and execute Python code to solve problems:

In [ ]:
#pragma warning disable MEAI001
#pragma warning disable OPENAI001
async Task Main()
{
    Console.WriteLine("=== Azure AI Agent with Code Interpreter Example ===");

    var agent = await aiProjectClient.CreateAIAgentAsync(
        name: "CodingAgent",
        instructions:
            "You are an expert Python programmer. Write clean, efficient code and explain your solutions.",
        model: modelDeploymentName!,
        tools: [new HostedCodeInterpreterTool() { Inputs = [] }]
    );

    var query = "Generate the factorial of 100 using python code, show the code and execute it.";
    await RunAgentWithQuery(agent, query);
}


## Execute the Example

Run the main function to see the code interpreter in action:

In [ ]:
await Main();
return 0;

## Try a more complex example

Let's test the agent with more complex programming tasks:


In [ ]:

Console.WriteLine("=== Azure AI Agent with Code Interpreter Example ===");

var agent = await aiProjectClient.CreateAIAgentAsync(
    name: "CodingAgent",
    instructions:
        "You are a helpful assistant that can write and execute code to answer user questions.",
    model: modelDeploymentName!,
    tools: [new HostedCodeInterpreterTool() { Inputs = [] }]
);

var query = new string[] { "Implement the Fibonacci sequence using recursion and iteration, compare their performance",
            "Create a simple data analysis of a randomly generated dataset with statistics and a plot",
            "Solve a quadratic equation ax² + bx + c = 0 for a=1, b=-5, c=6" };
foreach (var q in query)
{
    Console.WriteLine($"\n--- Running query: {q} ---\n");
    await RunAgentWithQuery(agent, q);
}

## Key Takeaways

1. **Code Interpreter Tool**: The `HostedCodeInterpreterTool` allows agents to execute Python code in a secure environment
2. **Mathematical Computation**: Perfect for complex calculations like factorials, statistics, and mathematical operations
3. **Data Analysis**: Can perform data analysis tasks including generating plots and statistics
4. **Code Visibility**: You can access the actual code executed through `ShowToolCallContent()`
5. **Streaming Support**: Works with both streaming and non-streaming responses
6. **Security**: Code execution happens in a secure, isolated environment
7. **Versatility**: Can handle various programming tasks from simple calculations to complex algorithms

## Best Practices

1. **Clear Instructions**: Provide clear instructions about what kind of code execution you need
2. **Error Handling**: The agent will handle code errors gracefully and suggest fixes
3. **Resource Management**: Use async context managers for proper cleanup
4. **Code Review**: Always review the executed code for understanding and validation
5. **Performance**: Be mindful of computational complexity for large-scale operations

## Use Cases

- Mathematical problem solving
- Data analysis and visualization
- Algorithm implementation and testing
- Educational coding assistance
- Prototype development
- Statistical computations